In [1]:
from facenet_pytorch import MTCNN

In [2]:
import numpy as np

In [3]:
from emotion.model.vgg19 import vgg19
import torch
import matplotlib.pyplot as plt


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


LABELS = {
    0 : 'Angry',
    1 : 'Disgust',
    2 : 'Fear', 
    3 : 'Happy', 
    4 : 'Sad', 
    5 : 'Surprise',
    6 : 'Neutral'
}


import os

model_dir = os.path.join(os.getcwd(),'emotion', 'data', 'saved', 'emotion.pth')
net = vgg19(1,7)
net.load_state_dict(torch.load(model_dir))
net = net.to(device)
net.eval()


vgg19(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [14]:
import cv2

# Load the cascade
mtcnn = MTCNN(thresholds = [0.7, 0.5, 0.5], device = 'cuda', margin=100)
# To capture video from webcam. 
cap = cv2.VideoCapture(0)
# To use a video file as input 
# cap = cv2.VideoCapture('filename.mp4')

def predict(img):
    #print(img.shape)
    img = cv2.resize(img, (48,48))
    img = torch.Tensor(img)
    img = img.view(-1,48,48)
    img = img.view(-1,1,48,48)
    img  = img.to(device)
    print(img.shape)
#     print(img.shape)
    o = net(img)
    print(o)
    pre = torch.max(o, 1)[1]
    print(f'predict {pre}')
    return LABELS[pre.item()]


def text(t, img, p):
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = p
    fontScale              = 1
    fontColor              = (0,0,255)
    lineType               = 2
    org                    = (0,100)

    cv2.putText(img,t, 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)

while True:
    # Read the frame
    _, img = cap.read()
    # Detect the faces
   
    faces = mtcnn.detect(img)
    print(faces)
    
    
    # Draw the rectangle around each face
    if (type(faces[0]) != type(None)):
        for (x, y, x1, y1) in faces[0]:
            x1 = int(x1)
            y1 = int(y1)
            x = int(x)
            y = int(y)
            
            print(x , y , x1 , y1 , sep = ' ')
            
            try:
                cv2.rectangle(img, (x, y), (x1, y1), (255, 0, 0), 2)
                face = img[y : y1, x: x1]
                gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                cv2.imwrite('test.jpg', gray)
                t = predict(gray)
                text(t,img, (x,y))
            except:
                print('error')
            

    cv2.imshow('img', img) 
                                  

    # Display
    # Stop if escape key is pressed
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
# Release the VideoCapture object
cap.release()

(array([[302.42267,  90.01985, 453.771  , 283.25305]], dtype=float32), array([0.98960847], dtype=float32))
302 90 453 283
torch.Size([1, 1, 48, 48])
tensor([[ 1.4053, -0.5833,  0.2403, -1.5425,  1.3745, -2.6884,  1.7847]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([6], device='cuda:0')
(array([[313.24106,  95.2213 , 463.1624 , 286.75262]], dtype=float32), array([0.98183674], dtype=float32))
313 95 463 286
torch.Size([1, 1, 48, 48])
tensor([[ 1.1571, -0.6396,  0.1816, -1.6402,  1.5469, -2.6406,  1.9545]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([6], device='cuda:0')
(None, array([None], dtype=object))
(array([[325.9977  , 102.801544, 479.78143 , 302.41797 ],
       [457.77896 , 367.27188 , 489.95944 , 406.49084 ]], dtype=float32), array([0.96473676, 0.68273836], dtype=float32))
325 102 479 302
torch.Size([1, 1, 48, 48])
tensor([[ 1.5521, -0.4258,  0.3696, -1.6485,  1.7622, -2.8906,  1.2399]],
       device='cuda:0', grad_fn=<AddmmBackward>)


(array([[323.4202 ,  89.67609, 485.02917, 293.8916 ]], dtype=float32), array([0.9712913], dtype=float32))
323 89 485 293
torch.Size([1, 1, 48, 48])
tensor([[ 1.6284, -0.1376,  0.9305, -2.2730,  1.1512, -1.9393,  0.7941]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([0], device='cuda:0')
(array([[323.68457,  90.29189, 484.66855, 291.33667]], dtype=float32), array([0.97281027], dtype=float32))
323 90 484 291
torch.Size([1, 1, 48, 48])
tensor([[ 1.5292, -0.1956,  0.8854, -2.3102,  1.1525, -1.9269,  1.0044]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([0], device='cuda:0')
(array([[322.63254,  87.93558, 484.444  , 291.46436]], dtype=float32), array([0.97368866], dtype=float32))
322 87 484 291
torch.Size([1, 1, 48, 48])
tensor([[ 1.6867, -0.2987,  0.5229, -1.9264,  1.3861, -2.6113,  1.2974]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([0], device='cuda:0')
(array([[320.8407 ,  87.18619, 484.03326, 293.0708 ]], dtype=float32), arr

(array([[346.966  , 122.17379, 447.3136 , 247.15999]], dtype=float32), array([0.6318237], dtype=float32))
346 122 447 247
torch.Size([1, 1, 48, 48])
tensor([[ 0.2006, -1.1203, -0.2709,  1.6270,  0.5785, -1.6032,  0.3743]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([3], device='cuda:0')
(array([[322.82825,  96.3943 , 483.12616, 293.84338]], dtype=float32), array([0.8871752], dtype=float32))
322 96 483 293
torch.Size([1, 1, 48, 48])
tensor([[ 0.9076, -0.9371,  0.0688,  0.1043,  1.0974, -2.3325,  1.0286]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([4], device='cuda:0')
(array([[319.16302,  97.15744, 472.40842, 288.01648]], dtype=float32), array([0.92746115], dtype=float32))
319 97 472 288
torch.Size([1, 1, 48, 48])
tensor([[ 0.7296, -1.2788, -0.3642,  1.1234,  0.9229, -2.5004,  1.1842]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([6], device='cuda:0')
(array([[320.9462  ,  92.266914, 475.88937 , 281.15503 ]], dtype=float32),

(None, array([None], dtype=object))
(None, array([None], dtype=object))
(array([[336.57446, 105.9886 , 489.98163, 296.45642]], dtype=float32), array([0.9621283], dtype=float32))
336 105 489 296
torch.Size([1, 1, 48, 48])
tensor([[ 0.9618, -0.5215,  0.1620, -1.3205,  1.4385, -2.3962,  1.5962]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([6], device='cuda:0')
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(array([[348.6214 , 107.40922, 487.15326, 283.1508 ]], dtype=float32), array([0.5138276], dtype=float32))
348 107 487 283
torch.Size([1, 1, 48, 48])
tensor([[ 0.5324, -0.6572,  0.2851, -1.0513,  1.2236, -1.8230,  1.4253]],


torch.Size([1, 1, 48, 48])
tensor([[ 1.6887, -0.3382,  1.4723, -3.9277,  1.8402, -2.0372,  1.4384]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([4], device='cuda:0')
(array([[534.23376, 446.28326, 558.7579 , 474.31375]], dtype=float32), array([0.5438965], dtype=float32))
534 446 558 474
torch.Size([1, 1, 48, 48])
tensor([[ 1.7527, -0.2183,  1.2793, -3.9257,  1.5205, -1.8619,  1.6074]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([0], device='cuda:0')
(array([[296.814  , 128.61748, 445.3284 , 312.76678]], dtype=float32), array([0.98238087], dtype=float32))
296 128 445 312
torch.Size([1, 1, 48, 48])
tensor([[ 1.3004, -1.0141, -0.4702, -0.1682,  1.6612, -3.4357,  1.9176]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([6], device='cuda:0')
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(array([[301.8888 , 130.1947 , 449.25665, 312.2737

...

torch.Size([1, 1, 48, 48])
tensor([[ 1.7894, -1.1345,  0.7703, -2.1525,  2.5980, -3.5806,  1.7614]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([4], device='cuda:0')
(array([[295.00623,  81.64278, 429.59677, 256.91443]], dtype=float32), array([0.9850552], dtype=float32))
295 81 429 256
torch.Size([1, 1, 48, 48])
tensor([[ 1.7866, -1.2860,  0.6267, -1.5050,  2.6678, -3.8047,  1.5282]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([4], device='cuda:0')
(array([[295.72244,  82.6871 , 428.3795 , 249.74684]], dtype=float32), array([0.9831472], dtype=float32))
295 82 428 249
torch.Size([1, 1, 48, 48])
tensor([[ 1.1065, -2.0227, -0.4271,  1.1006,  2.2434, -4.1360,  1.8211]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([4], device='cuda:0')
(array([[298.52026,  85.37972, 428.1539 , 247.53113]], dtype=float32), array([0.9822405], dtype=float32))
298 85 428 247
torch.Size([1, 1, 48, 48])
tensor([[ 1.2873, -1.7966,  0.0335, -0.0702,  2.5

(array([[318.98474, 129.31483, 427.53397, 264.4193 ]], dtype=float32), array([0.96004343], dtype=float32))
318 129 427 264
torch.Size([1, 1, 48, 48])
tensor([[ 1.4031, -0.5180,  0.6373, -1.7069,  1.6887, -2.3608,  0.8882]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([4], device='cuda:0')
(array([[316.8817 , 128.56029, 434.07202, 272.7799 ]], dtype=float32), array([0.9955799], dtype=float32))
316 128 434 272
torch.Size([1, 1, 48, 48])
tensor([[ 1.4593, -1.1439, -0.1894, -0.1989,  1.7117, -3.3936,  1.6178]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([4], device='cuda:0')
(array([[318.15906, 128.2212 , 435.1365 , 273.44687]], dtype=float32), array([0.9971113], dtype=float32))
318 128 435 273
torch.Size([1, 1, 48, 48])
tensor([[ 1.1158, -1.4066, -0.3936,  0.6357,  1.3513, -3.0558,  1.5766]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([6], device='cuda:0')
(array([[318.02682, 128.2512 , 432.76416, 271.75397]], dtype=float32), a

(array([[302.4149 ,  74.15001, 430.54684, 233.52434]], dtype=float32), array([0.9852771], dtype=float32))
302 74 430 233
torch.Size([1, 1, 48, 48])
tensor([[ 1.8963, -1.6881,  1.6997, -2.0454,  3.3660, -3.4975,  0.4679]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([4], device='cuda:0')
(array([[303.36508,  74.25859, 430.55554, 232.41252]], dtype=float32), array([0.98708147], dtype=float32))
303 74 430 232
torch.Size([1, 1, 48, 48])
tensor([[ 1.8739, -1.7063,  1.9489, -2.3044,  3.3533, -3.2044,  0.2688]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([4], device='cuda:0')
(array([[303.2372 ,  75.70507, 426.98822, 228.59991]], dtype=float32), array([0.9918486], dtype=float32))
303 75 426 228
torch.Size([1, 1, 48, 48])
tensor([[ 1.3838, -1.9431,  1.7437, -1.3097,  2.9990, -2.7624,  0.0474]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([4], device='cuda:0')
(array([[303.2372 ,  75.70507, 426.98822, 228.59991]], dtype=float32), arra

(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(array([[279.2597  , 120.701126, 434.1762  , 313.81274 ]], dtype=float32), array([0.96474177], dtype=float32))
279 120 434 313
torch.Size([1, 1, 48, 48])
tensor([[ 1.1609, -1.0397, -0.4025,  0.2475,  1.4539, -3.1667,  1.5475]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([6], device='cuda:0')
(array([[  0.58252645, 370.7413    ,  31.580015  , 403.99042   ]],
      dtype=float32), array([0.92108554], dtype=float32))
0 370 31 403
torch.Size([1, 1, 48, 48])
tensor([[-0.2404, -1.7240,  0.7549, -1.0348,  2.1578, -2.0551,  1.8628]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([4], device='cuda:0')
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(

(None, array([None], dtype=object))
(array([[264.60312 ,  89.671135, 417.4095  , 279.55072 ]], dtype=float32), array([0.98197913], dtype=float32))
264 89 417 279
torch.Size([1, 1, 48, 48])
tensor([[ 0.7877, -2.0630, -0.9747,  2.2624,  1.8306, -3.9219,  1.6207]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([3], device='cuda:0')
(None, array([None], dtype=object))
(None, array([None], dtype=object))
(array([[263.4174 ,  89.37309, 417.06238, 279.6813 ]], dtype=float32), array([0.98334587], dtype=float32))
263 89 417 279
torch.Size([1, 1, 48, 48])
tensor([[ 0.7586, -2.1809, -1.1076,  2.6200,  1.7392, -3.9825,  1.6561]],
       device='cuda:0', grad_fn=<AddmmBackward>)
predict tensor([3], device='cuda:0')
(array([[264.35248,  87.92368, 418.16394, 280.71722]], dtype=float32), array([0.9893272], dtype=float32))
264 87 418 280
torch.Size([1, 1, 48, 48])
tensor([[ 0.8797, -2.0888, -0.9961,  2.3334,  1.9722, -4.1680,  1.5883]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [12]:
type(None)

NoneType

NameError: name 'array' is not defined